In [4]:
import sys
sys.path.insert(0, '../python_scripts')
import json
import glob
import random
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq
import matplotlib.patches as patches
import numpy as np
from PIL import Image
import utils
import importlib
import threading
import meta
import heatmap
#metadata = utils.load("./variables/metadata.p")
from keras.optimizers import *
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras import callbacks
from keras.preprocessing.image import Iterator
import os
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
import heatmap
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
from shutil import copyfile
from keras.models import load_model
from keras.models import model_from_json
from keras.layers.advanced_activations import LeakyReLU
import time
import h_gen,meta
from meta import *
from h_gen import *
from utils import *
import gc 
import importlib

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN None)


In [19]:
image_size = [(576,1024),(1152,2048)]
FOLDER_TRAIN = "./temp_training/Resnet/" # Intermediate vectors
SPLIT = 0.8 # For the train/test split
importlib.reload(meta)
importlib.reload(h_gen)
metadata = meta.create_metadata()

['../JSONS/alb_labels.json', '../JSONS/bet_labels.json', '../JSONS/dol_labels.json', '../JSONS/lag_labels.json', '../JSONS/other_labels.json', '../JSONS/shark_labels.json', '../JSONS/yft_labels.json']



In [ ]:
metadata = meta.create_metadata()

In [20]:
for img_size in image_size:
    dim = str(img_size[0])+'_'+str(img_size[1])+'/'
    mk(FOLDER_TRAIN+''+dim)

##### Let's do some precomputing:

In [4]:
for img_size in image_size:
    print(img_size)

    dim = str(img_size[0])+'_'+str(img_size[1])+'/'
    model1 = VGG19(include_top=False, input_shape=(3,) + img_size)
    model2 = Model(model1.input, model1.layers[-2].output)

    img_a = Input(shape=(3,) + img_size)
    x = model2(img_a)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1,1), border_mode="valid")(x)
    x = AtrousConvolution2D(4096,7 , 7, atrous_rate=(2,2), border_mode='valid')(x)
    model = Model(img_a, x)

    del model1
    gc.collect()
    model3 = VGG19(include_top=True)    
    heatmap.insert_weights(model3.layers[-3], model.layers[-1])

    del model3
    gc.collect()
    
    X = np.zeros((1, 3)+img_size)
    f = os.listdir(FOLDER_TRAIN + dim)
    for key, v in tq(metadata.items()):
        if v["filename"]+ ".npy" not in f:
        # First of all, if there is no rectangle and the image is not in NoF, 
        # then there is a problem.

            if v["class"] != "NoF" and "rectangles" not in v:
                continue

            X[0] = img_to_array(load_img(v["path"],target_size=img_size))
            X = preprocess_input(X)
            y = model.predict(X)[0]
            utils.save_array(y, FOLDER_TRAIN +dim+ v["filename"])
    output_size = heatmap.get_dim(model, -1, input_shape=(3,)+img_size)
    utils.save(output_size, FOLDER_TRAIN + dim +"mask_size")
    del model1,model2,model
    gc.collect()
    

(576, 1024)


KeyboardInterrupt: 

## First step, create train and test set.

In [21]:
import operator
output_size = []
nb_filters = []
mask_size = []
def find_bigger(mask):
    obj = []
    for i,m in enumerate(mask):
        obj.append([i,m[0]*m[1]])
    return sorted(obj,key=operator.itemgetter(1),reverse=True)[0][0]
    
for i,img_size in enumerate(image_size):
    dim = str(img_size[0])+'_'+str(img_size[1])+'/'
    output_size.append(utils.load(FOLDER_TRAIN +dim+ "mask_size"))
    nb_filters.append(output_size[0][0])
    mask_size.append(output_size[i][1:])
    print("size of our filters: " + str(nb_filters[i]))
    print("size of our mask: " + str(mask_size[i]))
nb_filters = nb_filters[0]
print(mask_size)
print(nb_filters)

size of our filters: 2048
size of our mask: (18, 32)
size of our filters: 2048
size of our mask: (36, 64)
[(18, 32), (36, 64)]
2048


In [56]:
##THAT PART IS ONLY FOR ME . AS I HAVE ONLY 127 IMAGES IN (1152, 2048)
def filter_sample_to_same(list_files):
    import collections
    collection = collections.defaultdict(list)
    for list_file in list_files:
        for f in list_file:
            collection[f.split('/')[-1].split('.')[0]].append(f)
    return collection
def filter_collection(collection):
    obj = []
    for key in collection.keys():
        if len(collection[key]) > 1:
            obj.append(collection[key])
    return obj
        
def get_same_sample():
    list_files = []
    for img_size in image_size:    
        dim = str(img_size[0])+'_'+str(img_size[1])+'/'
        a = glob.glob(FOLDER_TRAIN +dim+ "*.npy")
        list_files.append([x.replace("\\", "/") for x in a])
    return list_files
print(get_same_sample())
collection = filter_collection(filter_sample_to_same(get_same_sample()))
print(len(collection))

[['./temp_training/Resnet/576_1024/img_00012.jpg.npy', './temp_training/Resnet/576_1024/img_00015.jpg.npy', './temp_training/Resnet/576_1024/img_00019.jpg.npy', './temp_training/Resnet/576_1024/img_00020.jpg.npy', './temp_training/Resnet/576_1024/img_00021.jpg.npy', './temp_training/Resnet/576_1024/img_00022.jpg.npy', './temp_training/Resnet/576_1024/img_00023.jpg.npy', './temp_training/Resnet/576_1024/img_00037.jpg.npy', './temp_training/Resnet/576_1024/img_00041.jpg.npy', './temp_training/Resnet/576_1024/img_00043.jpg.npy', './temp_training/Resnet/576_1024/img_00045.jpg.npy', './temp_training/Resnet/576_1024/img_00047.jpg.npy', './temp_training/Resnet/576_1024/img_00052.jpg.npy', './temp_training/Resnet/576_1024/img_00055.jpg.npy', './temp_training/Resnet/576_1024/img_00057.jpg.npy', './temp_training/Resnet/576_1024/img_00061.jpg.npy', './temp_training/Resnet/576_1024/img_00063.jpg.npy', './temp_training/Resnet/576_1024/img_00069.jpg.npy', './temp_training/Resnet/576_1024/img_00072.j

In [51]:
list_files = []
list_files = [x[0].split('/')[-1].split('.')[0]+'.jpg'  for x in collection]
random.shuffle(list_files)
l = int(0.8 * len(list_files))

training_set = list_files[:l]
test_set = list_files[l:]
print("First element for debug: " + training_set[0])

IndexError: list index out of range

## Second step, create the data generator.

In [8]:
for X, Y, W in DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, training_set, metadata,image_size):
    break

In [136]:
print("shape of X:", [x.shape for x in X])
print("lenght of Y:", len(Y))
print("shape of Y[0]:", Y[0].shape)
print("lenght of W:", len(W))
print("shape of W[0]:", W[0].shape)

shape of X: [(2, 2048, 18, 32), (2, 2048, 36, 64)]
lenght of Y: 5
shape of Y[0]: (2, 2304, 1)
lenght of W: 5
shape of W[0]: (2, 2304)


In [130]:
train_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, training_set, metadata,image_size)
test_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, test_set, metadata,image_size)

In [10]:
def img_mask_generator(batch_size, nb_filters, mask_size, folder, file_list):
    reshaped = mask_size[0]*mask_size[1]
    
    X = np.zeros((batch_size, nb_filters)+mask_size)
    Y = [np.zeros((batch_size, reshaped,1)) for _ in range(5)]
    W = [np.zeros((batch_size, reshaped))   for _ in range(5)]
    
    
    while True:

        selected_files = random.sample(file_list, batch_size)
        meta = [metadata[x] for x in selected_files]
        
        # We loop over the files
        for c, v in enumerate(meta):
            
            masks, m_weights = h_gen.get_5_masks_w(v, mask_size)
            
            m_weights = np.expand_dims(m_weights, axis=0)
            m_weights = np.reshape(m_weights,(reshaped,))
            
            # loop over the 5 masks
            for i, mask in enumerate(masks):
                
                mask0 = np.expand_dims(mask, axis=0)
                mask0 = np.reshape(mask0,(reshaped,))
                y = np.expand_dims(mask0, axis=1)

                X[c] = utils.load_array(folder + v["filename"])
                Y[i][c] = y
                W[i][c] = m_weights

        yield X, Y, W
            

## Third step, create the network.

##### First, let's create the network that will be shared:

In [8]:
nb_input_filters = 1000
print(mask_size)
print(nb_filters)
# We put this value so that we can reuse the weights of the final layer of the resnet.

[(18, 32), (36, 64)]
2048


In [30]:
def create_separate_models(nb_input_filters, mask_size):

    img_a = Input(shape=(nb_input_filters,) + mask_size)

    x = Convolution2D(512, 1, 1, activation='relu')(img_a)
    x = Convolution2D(256, 1, 1, activation='relu')(x)
    x = Convolution2D(128, 1, 1, activation='relu')(x)
    x = Convolution2D(32, 1, 1, activation='relu')(x)
    x = Convolution2D(16, 1, 1, activation='relu')(x)
    x = Convolution2D(1, 1, 1, activation='linear')(x)
    #x = LeakyReLU(alpha=0.3)(x)
    x = Reshape((1, mask_size[0]*mask_size[1]))(x)
    x = Permute((2, 1))(x)

    return Model(img_a, x)

In [31]:
def model_custom_1(nb_input_filters):
    img_input = Input(shape=(nb_input_filters,) + (None,None)) 
    x = Convolution2D(512, 1, 1, activation='relu')(img_input)
    x = Convolution2D(256, 1, 1, activation='relu')(x)
    x = Convolution2D(128, 1, 1, activation='relu')(x)
    last_filter = 64
    x = Convolution2D(last_filter, 1, 1, activation='relu')(x)
    return Model(input=img_input,output=x), last_filter

def model_custom_2(nb_input_filters):
    a,b = tuple([Input(shape=(nb_input_filters,) + m) for m in mask_size])

    c = Convolution2D(16, 1, 1, activation='relu')(a)
    #c = BatchNormalization(axis=1)(c)
    c = Convolution2D(16, 1, 1, activation='relu')(c)
    #c = BatchNormalization(axis=1)(c)
    c = UpSampling2D(size=(2, 2), dim_ordering='default')(c)
    
    d = Convolution2D(16, 1, 1, activation='relu')(b)
    #d = BatchNormalization(axis=1)(d)
    d = Convolution2D(16, 1, 1, activation='relu')(d)
    #d = BatchNormalization(axis=1)(d)
    
    e = Merge(mode='concat', concat_axis=1)([c,d])
    #e = BatchNormalization(axis=1)(e)
    e = Convolution2D(16, 1, 1, activation='relu')(e)
    #e = BatchNormalization(axis=1)(e)
    #e = Convolution2D(16, 1, 1, activation='relu')(e)
    #e = BatchNormalization(axis=1)(e)
    e = Convolution2D(1, 1, 1, activation='linear')(e)
    #e = BatchNormalization(axis=1)(e)
    e = Reshape((1, mask_size[1][0]*mask_size[1][1]))(e)
    e = Permute((2, 1))(e)
    
    return Model(input=[a,b],output=e)



##### Here is the final network:

In [32]:
nb_input_filters = 1000
img_inputs = [Input(shape=(nb_filters,) + m) for m in mask_size]
print(img_inputs)

first_con = Convolution2D(nb_input_filters, 1, 1, activation='relu')
x, y = [first_con(inp) for inp in img_inputs]

x = BatchNormalization(axis=1)(x)
y = BatchNormalization(axis=1)(y)

W = []
for _ in range(5):
    model_cus_1, last_filter = model_custom_1(nb_input_filters)
    w = model_cus_1(x)
    z = model_cus_1(y)
    W.append((w,z))
    
Z=[]
for a,b in W:
    model_cus_2 = model_custom_2(last_filter)
    k = model_cus_2([a,b])
    Z.append(k)
    
model = Model(input=img_inputs, output=Z)



[input_18, input_19]


In [33]:
model.output

[InplaceDimShuffle{0,2,1}.0,
 InplaceDimShuffle{0,2,1}.0,
 InplaceDimShuffle{0,2,1}.0,
 InplaceDimShuffle{0,2,1}.0,
 InplaceDimShuffle{0,2,1}.0]

##### Let's insert the imageNet weights:

In [34]:
# We need the last weights of the resnet50:
resnet = ResNet50()

In [35]:
model.layers[2]

In [36]:
resnet.layers[-1]

In [37]:
heatmap.insert_weights(resnet.layers[-1], model.layers[2])

In [38]:
model_file = "models/resnet_mask_training.h5"
mk('models/')

In [39]:
# we can now save the model.
model.save(model_file)

## Fourth step, training.

In [28]:
model_file = "models/fish_detection_trained_1.1.h5"
model = load_model(model_file)

In [41]:
#model.summary()

In [42]:
model.layers[2].trainable=False

In [43]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.0, nesterov=True)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=adam, sample_weight_mode="temporal")

In [44]:
train_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, training_set, metadata,image_size)
test_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, test_set, metadata,image_size)
history = model.fit_generator(train_gen, samples_per_epoch=2, nb_epoch=100, callbacks=[remote], 
                              verbose=0,validation_data=test_gen, nb_val_samples=2)
# 

In [59]:
model.layers[2].trainable=True

In [60]:
sgd = SGD(lr=0.00001, decay=1e-6, momentum=0.0, nesterov=True)
adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=sgd, sample_weight_mode="temporal")

In [61]:
train_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, training_set, metadata,image_size)
test_gen = DiskArrayIterator(2, nb_filters, mask_size, FOLDER_TRAIN, test_set, metadata,image_size)
history = model.fit_generator(train_gen, samples_per_epoch=2, nb_epoch=100, callbacks=[remote], 
                              verbose=0,validation_data=test_gen, nb_val_samples=2)
# 

In [1]:
file_trained = "models/fish_detection_trained1.1.h5"

In [2]:
model.save(file_trained)

NameError: name 'model' is not defined

## Fifth step, display results.

In [47]:
class DiskArrayIterator(Iterator):
    
    def __init__(self, batch_size, nb_filters, mask_size, folder, file_list, 
                 metadata,image_size, with_name = False, shuffle=False, seed=None):
        self.batch_size=batch_size
        self.nb_filters=nb_filters
        self.mask_size=mask_size
        self.folder=folder
        self.file_list=file_list
        self.metadata = metadata
        self.with_name = with_name
        self.image_size = image_size
        super(DiskArrayIterator, self).__init__(len(file_list), batch_size, shuffle, seed)
    
    def find_bigger(mask):
        obj = []
        for i,m in enumerate(mask):
            obj.append([i,m[0]*m[1]])
        return sorted(obj,key=operator.itemgetter(1),reverse=True)[0][0]

    def next(self):
        with self.lock:
            try:
                index_array, current_index, current_batch_size = next(self.index_generator)
            except:
                index_array = self.batch_size
        mask_larger = self.mask_size[find_bigger(self.mask_size)]
        reshaped = mask_larger[0]*mask_larger[1]
        X = [np.zeros((self.batch_size, self.nb_filters) + m) for m in self.mask_size]
        Y = [np.zeros((self.batch_size, reshaped,1)) for _ in range(5)]
        W = [np.zeros((self.batch_size, reshaped))   for _ in range(5)]
        selected_files = [self.file_list[i] for i in range(index_array)]
        meta = [metadata[x] for x in selected_files]
        # We loop over the files
        for c, v in enumerate(meta):
            
            masks, m_weights = h_gen.get_5_masks_w(v, mask_larger)
            
            m_weights = np.expand_dims(m_weights, axis=0)
            m_weights = np.reshape(m_weights,(reshaped,))
            
            # loop over the 5 masks
            for i, mask in enumerate(masks):
                
                mask0 = np.expand_dims(mask, axis=0)
                mask0 = np.reshape(mask0,(reshaped,))
                y = np.expand_dims(mask0, axis=1)
                for j,image_size in enumerate(self.image_size):
                    dim = str(image_size[0])+'_'+str(image_size[1])+'/'
                    X[j][c] = utils.load_array(self.folder +dim+v["filename"])
                Y[i][c] = y
                W[i][c] = m_weights
        
        if self.with_name:
            return X, Y, W, [v["path"] for v in meta]
        return X, Y, W
        #return X, Y

[]

In [48]:
for X, Y, W in DiskArrayIterator(20, nb_filters, mask_size, FOLDER_TRAIN, training_set, metadata,image_size):
    break

IndexError: list index out of range

In [43]:
model_file = "models/fish_detection_trained_1.1.h5"
model = load_model(model_file)

In [ ]:
Z = model.predict(X)

In [ ]:
for i in range(4,8):
    print(paths[i])
    plt.imshow(load_img(paths[i]))
    plt.show()
    for j in range(5):
        plt.figure(figsize=(24,24))
        plt.subplot(1,2,1)
        plt.imshow(np.reshape(Y[j][i],mask_size[1]))
        plt.subplot(1,2,2)
        plt.imshow(np.reshape(Z[j][i],mask_size[1]))
        plt.show()
        print("max: ", np.max(Z[j][i]))
        print("min: ", np.min(Z[j][i]))

In [ ]:
for X, Y, W , paths in DiskArrayIterator(16, nb_filters, mask_size, FOLDER_TRAIN, test_set, metadata,image_size):
    break

In [ ]:
Z = model.predict(X)

In [ ]:
for i in range(4):
    print(paths[i])
    plt.imshow(load_img(paths[i]))
    plt.show()
    for j in range(5):
        plt.figure(figsize=(24,24))
        plt.subplot(1,2,1)
        plt.imshow(np.reshape(Y[j][i],mask_size[1]))
        plt.subplot(1,2,2)
        plt.imshow(np.reshape(Z[j][i],mask_size[1]))
        plt.show()
        print("max: ", np.max(Z[j][i]))
        print("min: ", np.min(Z[j][i]))

In [ ]:
def img_mask_generator(batch_size=32, image_size = (224,224),mask_size=(100,100), lambda_ratio=10):
    reshaped = mask_size[0]*mask_size[1]
    
    X = np.zeros((batch_size, 3)+image_size)
    #X = np.ones((batch_size, 1)+mask_size)
    #Y = np.zeros((batch_size, mask_size[0]*mask_size[1], 2))
    Y = np.zeros((batch_size, reshaped,1))
    W = np.zeros((batch_size, reshaped))
    while True:
        c=0
        for key, v in metadata.items():
            
            # First of all, if there is no rectangle and the image is not in NoF, 
            # then there is a problem.
            
            if v["class"] != "NoF" and "rectangles" not in v:
                continue
            mask0 = np.expand_dims(h_gen.get_mask(v, mask_size), axis=0)
            mask0 = np.reshape(mask0,(reshaped,))
            
            y = np.expand_dims(mask0, axis=1)

            X[c] = img_to_array(load_img(v["path"],target_size=image_size))
            Y[c] = y
            W[c] = mask0*(lambda_ratio-1) + np.ones(mask0.shape)
            c+=1
            
            if c == batch_size:
                c = 0 
                yield X, Y, W
            

In [ ]:
batch_size = 2
image_size = (720,1280)
lambda_ratio = 20

for X, Y, W in img_mask_generator(batch_size, image_size,(100,100), lambda_ratio):
    break

In [ ]:
batch_size = 2

lambda_ratio = 20

In [ ]:
model1 = ResNet50(include_top=False)
for layer in model1.layers:
    layer.trainable=False

In [ ]:
model1 = heatmap.to_heatmap(model1)

In [ ]:
for layer in model1.layers:
    layer.trainable=False

In [ ]:
model1.layers[-2]

In [ ]:
a,b,c = heatmap.get_dim(model1,-2, input_shape=(3,) + image_size)

In [ ]:
b,c

In [ ]:
a

In [ ]:
mask_size = (23, 40)

In [ ]:
mask_size = (b,c)

In [ ]:
img_a = Input(shape=(3,) + image_size)
model2 = Model(model1.input, model1.layers[-2].output)
x = model2(img_a)
x = AveragePooling2D(pool_size=(7, 7), strides=(1,1), border_mode="same")(x)
x = Convolution2D(1024, 1, 1, activation='relu')(x)
x = Convolution2D(512, 1, 1, activation='relu')(x)
x = Convolution2D(256, 1, 1, activation='relu')(x)
x = Convolution2D(128, 1, 1, activation='relu')(x)
x = Convolution2D(32, 1, 1, activation='relu')(x)
x = Convolution2D(16, 1, 1, activation='relu')(x)
x = Convolution2D(1, 1, 1, activation='linear')(x)
#x = LeakyReLU(alpha=0.3)(x)
x = Reshape((1, b*c))(x)
x = Permute((2, 1))(x)
model = Model(img_a,x)

In [ ]:
model2.summary()

In [ ]:
model.summary()

In [ ]:
sgd = SGD(lr=0.000001, decay=1e-6, momentum=0.0, nesterov=True)
model.compile(loss='mse', optimizer=sgd, sample_weight_mode="temporal")

In [ ]:
history = model.fit_generator(img_mask_generator(batch_size, image_size,mask_size, lambda_ratio), 
                    samples_per_epoch=8, nb_epoch=500, callbacks=[remote], verbose=0)

In [ ]:
for X, Y, W in img_mask_generator(batch_size, image_size,mask_size, lambda_ratio):
    Z = model.predict(X)
    break

In [ ]:
plt.imshow(array_to_img(X[0]))
plt.show()
plt.imshow(np.swapaxes(np.reshape(Y[0],(c,b)),0,1))
plt.show()
plt.imshow(np.swapaxes(np.reshape(Z[0],(c,b)),0,1) - np.min(Z[0]))
plt.show()
a = Z[0]
idx = Z[0][:,:] > 1
a[idx] = 1
idx = Z[0][:,:] < 0
a[idx] = 0 
plt.imshow(np.swapaxes(np.reshape(a,(c,b)),0,1))
plt.show()


In [ ]:
np.max(Z[0])

In [ ]:
np.min(Z[0])

In [ ]:
img_a = Input(shape=(3,) + image_size)
model2 = Model(model1.input, model1.layers[-2].output)
x = model2(img_a)
x = AveragePooling2D(pool_size=(7, 7), strides=(1,1), border_mode="same")(x)
model = Model(img_a,x)

In [ ]:
X = np.zeros((1, 3)+image_size)
for key, v in metadata.items():
    
    # First of all, if there is no rectangle and the image is not in NoF, 
    # then there is a problem.

    if v["class"] != "NoF" and "rectangles" not in v:
        continue

    #X[0] = img_to_array(load_img(v["path"],target_size=image_size))
    #y = model.predict(X)[0]
    y = utils.load("out_resnet/" + v["filename"])
    utils.save_array(y, "out_resnet/bco/" + v["filename"])

In [ ]:
def img_mask_generator(batch_size=32, image_size = (224,224),mask_size=(100,100), lambda_ratio=10, buffer_size=100):
    reshaped = mask_size[0]*mask_size[1]
    buffer = np.zeros((buffer_size, 2048) + mask_size)
    buffer_names = {}
    X = np.zeros((batch_size, 2048) + mask_size)
    Y = np.zeros((batch_size, 2048) + mask_size)
    #X = np.ones((batch_size, 1)+mask_size)
    #Y = np.zeros((batch_size, mask_size[0]*mask_size[1], 2))
    #Y = np.zeros((batch_size, reshaped,1))
    W = np.zeros((batch_size, reshaped))
    buffer_count = 0
    while True:
        c=0
        t = time.time()
        for key, v in metadata.items():
            
            # First of all, if there is no rectangle and the image is not in NoF, 
            # then there is a problem.
            
            if v["class"] != "NoF" and "rectangles" not in v:
                continue
            """mask0 = np.expand_dims(h_gen.get_mask(v, mask_size[::-1]), axis=0)
            mask0 = np.reshape(mask0,(reshaped,))
            #print(v["class"], v["filename"])
            y = np.expand_dims(mask0, axis=1)"""
            try:
                idx = buffer_names[v["filename"]]
                x = buffer[idx]
            except KeyError:
                x = utils.load_array("out_resnet/bco/" + v["filename"])
            
            if buffer_count < buffer_size:
                buffer[buffer_count] = x
                buffer_names[v["filename"]] = buffer_count
                buffer_count+=1
            
            X[c] = x
            Y[c] = x
            #W[c] = mask0*(lambda_ratio-1) + np.ones(mask0.shape)
            c+=1
            
            if c == batch_size:
                c = 0 
                yield X, Y
                #yield X, Y, W
        print(time.time() - t)

In [ ]:
img_a = Input(shape=(2048,) + mask_size)
x = Convolution2D(1024, 1, 1, activation='relu')(img_a)
x = Convolution2D(512, 1, 1, activation='relu')(x)
x = Convolution2D(1024, 1, 1, activation='relu')(x)
x = Convolution2D(2048, 1, 1, activation='linear')(x)
model = Model(img_a,x)

In [ ]:
sgd = SGD(lr=15, decay=0.001, momentum=0.0, nesterov=True)
model.compile(loss='mse', optimizer=sgd, sample_weight_mode="temporal")

In [ ]:
history = model.fit_generator(img_mask_generator(32, image_size,mask_size, lambda_ratio), 
                    samples_per_epoch=128, nb_epoch=5000, callbacks=[remote], verbose=0)

In [ ]:
def img_mask_generator(batch_size=32, image_size = (224,224),mask_size=(100,100), lambda_ratio=10, buffer_size=100):
    reshaped = mask_size[0]*mask_size[1]
    buffer = np.zeros((buffer_size, 2048) + mask_size)
    buffer_names = {}
    X = np.zeros((batch_size, 2048) + mask_size)
    #X = np.ones((batch_size, 1)+mask_size)
    #Y = np.zeros((batch_size, mask_size[0]*mask_size[1], 2))
    Y = np.zeros((batch_size, reshaped,1))
    W = np.zeros((batch_size, reshaped))
    buffer_count = 0
    while True:
        c=0
        t = time.time()
        for key, v in metadata.items():
            
            # First of all, if there is no rectangle and the image is not in NoF, 
            # then there is a problem.
            
            if v["class"] != "NoF" and "rectangles" not in v:
                continue
            """mask0 = np.expand_dims(h_gen.get_mask(v, mask_size[::-1]), axis=0)
            mask0 = np.reshape(mask0,(reshaped,))
            #print(v["class"], v["filename"])
            y = np.expand_dims(mask0, axis=1)"""
            try:
                idx = buffer_names[v["filename"]]
                x = buffer[idx]
            except KeyError:
                x = utils.load_array("out_resnet/bco/" + v["filename"])
            
            if buffer_count < buffer_size:
                buffer[buffer_count] = x
                buffer_names[v["filename"]] = buffer_count
                #buffer_count+=1
            
            X[c] = x
            #Y[c] = y
            #W[c] = mask0*(lambda_ratio-1) + np.ones(mask0.shape)
            c+=1
            
            if c == batch_size:
                c = 0 
                yield X, Y, W
        print(time.time() - t)

In [ ]:
for X, Y, W in img_mask_generator(8, image_size,mask_size, lambda_ratio):
    Z = model.predict(X)
    break

In [ ]:
mask_size = (23,40)

In [ ]:
for i in range(0,8):
    plt.imshow(np.reshape(Y[i],(b,c)))
    plt.show()
    print("max: " + str(np.max(Z[i])))
    print("min: " + str(np.min(Z[i])))
    print("loss= " + str(np.sum(np.square(Y[i]-Z[i])*Y[i]*(lambda_ratio-1) + np.ones(Y[i].shape))/(23*40)))
    plt.imshow(np.reshape(Z[i],(b,c)))
    plt.show()

In [ ]:
np.min(Z)

In [ ]:
mask_size = (23,40)

In [ ]:
b,c = 23,40

In [ ]:
img_a = Input(shape=(2048,) + mask_size)
x = Convolution2D(1024, 1, 1, activation='relu')(img_a)
x = Convolution2D(512, 1, 1, activation='relu')(x)
x = Convolution2D(256, 1, 1, activation='relu')(x)
x = Convolution2D(128, 1, 1, activation='relu')(x)
x = Convolution2D(32, 1, 1, activation='relu')(x)
x = Convolution2D(16, 1, 1, activation='relu')(x)
x = Convolution2D(1, 1, 1, activation='linear')(x)
#x = LeakyReLU(alpha=0.3)(x)
x = Reshape((1, b*c))(x)
x = Permute((2, 1))(x)
model = Model(img_a,x)

In [ ]:
sgd = SGD(lr=0.001, decay=0.001, momentum=0.0, nesterov=True)
model.compile(loss='mse', optimizer=sgd, sample_weight_mode="temporal")

In [ ]:
history = model.fit_generator(img_mask_generator(64, image_size,mask_size, lambda_ratio), 
                    samples_per_epoch=128, nb_epoch=5000, callbacks=[remote], verbose=0)

In [ ]:
np.sum(Z[0])

In [ ]:
np.all(np.equal(Y[0], Z[0]))

In [ ]:
np.sum(Z[0])

In [ ]:
img_input = Input(shape=(1,100,100))
x = Convolution2D(1, 1, 1, border_mode="same")(img_input)
#x = Permute((1,2))(x)
#x = Permute((1,3))(x)
x = Reshape((1, 100*100))(x)
x = Permute((2, 1))(x)
#x = heatmap.Softmax4D(axis=1,name="softmax")(x)
model = Model(img_input,x)

In [ ]:
model.layers[1].trainable = True

In [ ]:
a = model.layers[1].get_weights()

In [ ]:
a[1][0] = 0

In [ ]:
a[0][0,0,0,0] = 1

In [ ]:
model.layers[1].set_weights(a)

In [ ]:
model.layers[1].get_weights()

In [ ]:
model.layers[1].get_config()

In [ ]:
model.summary()

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.0, nesterov=True)
model.compile(loss='mse', optimizer=sgd, sample_weight_mode="temporal")

In [ ]:
model.fit_generator(img_mask_generator(batch_size=3000), samples_per_epoch=32, nb_epoch=20, callbacks=[remote], verbose=0)

In [ ]:
Z = model.predict(X)

In [ ]:
np.max(Z[0])